In [1]:
pip install scipy


Note: you may need to restart the kernel to use updated packages.


In [2]:
import scipy.io
import numpy as np

# Ruta del archivo .mat
mat_file_path = 'c:/Users/Adrian/Downloads/trainSet/FIXATIONLOCS/Action/001.mat'

# Cargamos el archivo .mat
mat_data = scipy.io.loadmat(mat_file_path)

# Imprimimos las claves disponibles en el archivo para entender qué contiene
print(mat_data.keys())

# Supongamos que el archivo contiene una matriz con coordenadas de fijación bajo la clave 'fixations'
# Accedemos a la matriz de fijaciones usando la clave 'fixLocs'
fixations = mat_data.get('fixLocs')

# Imprimimos la forma de la matriz y algunas filas de ejemplo para explorar el contenido
print(f"Forma de la matriz de fijaciones: {fixations.shape}")
print(f"Ejemplo de las primeras filas de coordenadas de fijación:\n{fixations[:5]}")



dict_keys(['__header__', '__version__', '__globals__', 'fixLocs'])
Forma de la matriz de fijaciones: (1080, 1920)
Ejemplo de las primeras filas de coordenadas de fijación:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Ruta de la imagen original (correspondiente al archivo de fijaciones)
image_path = 'C:/Users/Adrian/Downloads/trainSet/Stimuli/Action/001.jpg'

# Cargamos la imagen con OpenCV
image = cv2.imread(image_path)

# Convertimos la imagen de BGR a RGB para visualizarla correctamente con Matplotlib
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Creamos una figura con 3 subplots
fig, ax = plt.subplots(1, 3, figsize=(20, 10))

# Subplot 1: Imagen original
ax[0].imshow(image_rgb)
ax[0].set_title('Imagen Original')
ax[0].axis('on')  # Ocultar ejes

# Subplot 2: Imagen con los puntos de fijación
ax[1].imshow(image_rgb)
fixation_coords = np.argwhere(fixations == 1)
for coord in fixation_coords:
    ax[1].scatter(coord[1], coord[0], c='red', s=10)  # coord[1] es X y coord[0] es Y
ax[1].set_title('Imagen con Fijaciones')
ax[1].axis('on')  # Ocultar ejes

# Subplot 3: Solo fijaciones
ax[2].imshow(np.zeros_like(image_rgb))  # Fondo negro
for coord in fixation_coords:
    ax[2].scatter(coord[1], coord[0], c='red', s=10)  # coord[1] es X y coord[0] es Y
ax[2].set_title('Solo Fijaciones')
ax[2].axis('on')  # Ocultar ejes

# Mostrar la figuraa
plt.show()


# Preprocessing

In [ ]:
# Obtener el tamaño de la imagen (ancho y alto)
image_height, image_width, _ = image_rgb.shape

# Normalizamos las coordenadas de fijación dividiendo por el ancho y alto de la imagen
fixation_coords_normalized = fixation_coords.copy()
fixation_coords_normalized[:, 0] = fixation_coords[:, 0] / image_height  # Normalizamos Y
fixation_coords_normalized[:, 1] = fixation_coords[:, 1] / image_width   # Normalizamos X

# Mostramos algunos ejemplos de las coordenadas normalizadas
print(f"Ejemplo de coordenadas normalizadas:\n{fixation_coords_normalized[:5]}")


In [ ]:
plt.hist(fixation_coords[:, 0], bins=50, alpha=0.5, label='Y-coordinates')
plt.hist(fixation_coords[:, 1], bins=50, alpha=0.5, label='X-coordinates')
plt.legend(loc='upper right')
plt.title('Distribución de las Coordenadas de Fijación')
plt.show()


In [ ]:
# Aumentamos más el tamaño de los puntos de fijación para que sean más visibles
# En lugar de solo aumentar un rango pequeño, hacemos un área mayor alrededor de cada fijación
fixation_mask = np.zeros((image_height, image_width), dtype=np.float32)

# Aumentamos el rango de píxeles alrededor de cada coordenada de fijación
for coord in fixation_coords:
    y, x = coord[0], coord[1]
    fixation_mask[y-5:y+5, x-5:x+5] = 1  # Aumenta el rango de -5 a +5 píxeles

# Mostramos la máscara de fijaciones con puntos más grandes
plt.figure(figsize=(5, 5))
plt.imshow(fixation_mask, cmap='gray')
plt.title('Máscara de Fijaciones (Puntos grandes)')
plt.axis('off')
plt.show()


In [ ]:
import scipy.ndimage

# Creamos un mapa de calor vacío del tamaño de la imagen
heatmap = np.zeros((image_height, image_width), dtype=np.float32)

# Agregamos las fijaciones al mapa de calor, aumentando el valor en las coordenadas de fijación
for coord in fixation_coords:
    y, x = coord[0], coord[1]
    heatmap[y, x] += 1  # Incrementamos el valor en cada punto de fijación

# Aplicamos una convolución gaussiana para suavizar el mapa de calor (crear un efecto de "difusión")
heatmap = scipy.ndimage.gaussian_filter(heatmap, sigma=15)  # Ajusta sigma para suavizar más o menos

# Normalizamos el mapa de calor para que esté entre 0 y 1
heatmap /= np.max(heatmap)

# Mostramos el mapa de calor
plt.figure(figsize=(5, 5))
plt.imshow(heatmap, cmap='hot')  # Colormap "hot" para representar el calor
plt.title('Mapa de Calor de Fijaciones')
plt.axis('off')
plt.show()


In [ ]:
import numpy as np

# Supongamos que ya cargaste la matriz de fijaciones desde el archivo .mat
# Usamos la variable 'fixations' como ejemplo de la matriz cargada

# Obtenemos los valores únicos de la matriz
valores_unicos = np.unique(fixations)

# Imprimimos los valores únicos
print(f"Valores únicos en la matriz: {valores_unicos}")

# Verificamos si la matriz contiene únicamente 0s y 1s
if np.array_equal(valores_unicos, [0, 1]):
    print("La matriz solo contiene 0s y 1s (es binaria).")
else:
    print("La matriz contiene otros valores además de 0s y 1s.")
